In [ ]:
import pandas as pd
import numpy as np
import pywhatkit as wpp
from datetime import datetime 

#SetOptions para ver los dataframes
#pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth',False)

Lectura del Excel

In [ ]:
path = 'Data/'
mes = datetime.now().month

extras = pd.read_excel(path + "Román - M{} Seguimiento.xlsx".format(mes), sheet_name = "Extras")
database = pd.read_excel(path + "BASE CNCI.xlsx", sheet_name = 'Activos')


In [ ]:
extras.columns

Limpieza de Columnas

In [ ]:
extras = extras[["Matricula","Nombre","Calificacion", "Materia","Clave Carrera", "Tipo Calificacion", "Centro Enlace"]]
extras.shape

Filtran solo de Cadereyta y CC Cadereyta

In [ ]:
extras = extras[extras["Centro Enlace"].isin(["CADEREYTA", "CC CADEREYTA"])]

Se agrupan en una nueva columna las materias del alumno

Se Elimina la fila de Materias al final ya que pasa al frente.
Filtro de duplicados. Renombra la columna Materia por Materia.

In [ ]:
df_agrupado = extras.groupby("Nombre")["Materia"].agg(lambda x: ", ".join(x)).reset_index().rename(columns = {"Materia": "Materias"})
extras = pd.merge(extras, df_agrupado, on = "Nombre", how = "left")     

#Cambio de columna y eliminar la extra
columna_extra = extras["Materias"]
extras.drop(columns = ["Materias"], inplace=True)
extras["Materia"] = columna_extra

#Filtra dúplicados
extras = extras.drop_duplicates( subset = "Nombre")
#Renombra
extras.rename(columns = {"Materia" : "Materias"}, inplace= True)

Se eliminan todas las columnas después de la columna CELULAR. 
Se elimina la columna TUTOR ya que está vacía.
Cambia el formato de las columnas TELÉFONO y CELULAR

In [ ]:
#Se elimina USUARIO BB,PASSWORD PORTAL ALUMNO, PROYECTO, MODELO EDUCATIVO, SERVICIO, Modalidad, Monto, Correo 2
database.drop(columns = ["USUARIO BB","PASSWORD PORTAL \nALUMNO", 'PROYECTO','MODELO EDUCATIVO', 'Fecha', 'SERVICIO', 'Fecha de inicio','Modalidad', 'Monto', 'Correo 2'], inplace=True)

#Renombrar el nombre de la columna NOMBRE
database.rename(columns = {'NOMBRE                                                                                                                                      ' : "NOMBRE"}, inplace= True)
database.rename(columns = {'MATRICULA' : "Matricula"}, inplace= True)
#Cambia el formato de las columnas TELÉFONO y CELULAR
#df02_final["TELÉFONO"] = df02_final["TELÉFONO"].astype(str).apply(lambda x: "{:.0f}".format(float(x)))
#df02_final["CELULAR"] = df02_final["CELULAR"].astype(str).apply(lambda x: "{:.0f}".format(float(x)))
database

# Filtro Cadereyta
database = database[database['CE CANALIZADO'] == 'CE CADEREYTA']

Lista de los NOMBRES y MATRICULAS de los Reprobados

In [ ]:
matricula_reprobados = extras['Matricula'].unique()
matricula_reprobados

Reprobados que están en el Módulo 4

In [ ]:
extras.columns

Tabla comprendida en la cual solo está la información de los reprobados que están en Módulo 4

In [ ]:
reprobados_mod4 = database[(database['Matricula'].isin(matricula_reprobados)) & (database['M4'] == 4)]
reprobados_mod4 = pd.merge(extras, reprobados_mod4, on = 'Matricula')
reprobados_mod4.columns

In [ ]:
reprobados_mod4 = reprobados_mod4[['Matricula', 'Nombre', 'Calificacion', 'Materias', 'Clave Carrera', 'Tipo Calificacion', 'Centro Enlace', 'M4', 'Correo', 'Teléfono', 'Teléfono2', 'Asesor Académico', 'Asesor Financiero ', 'Asesor Escolar']]

In [ ]:
reprobados_mod4.shape

#### Archivos de utilidad

In [ ]:
path2 = 'infodata/'
try: 
    matric_error = pd.read_csv(path2 + 'matriculas_error.csv')
    matric_yacontactadas = pd.read_csv(path2 + 'matriculas_yacontactadas.csv')
    
    matric_error = matric_error.dropna().values.flatten().tolist()
    matric_yacontactadas = matric_yacontactadas.dropna().values.flatten().tolist()
except:
    matric_error = pd.DataFrame
    matric_yacontactadas = pd.DataFrame
    
print(matric_error)

Mensaje predeterminado para alumnos

In [ ]:
for index, row in reprobados_mod4.iterrows():
    alumno = row['Nombre']
    matricula = row['Matricula']
    tipoExamen = row['Tipo Calificacion']
    materias = row['Materias']
    telefono = row['Teléfono']
    print(telefono)
    if not np.isnan(telefono):
        telefono = '+52 '+str(telefono)
    if matricula in (matric_error):
        if not np.isnan(row['Teléfono2']):
            telefono2 ='+52 '+str(row['Teléfono2'])
            # print(telefono, telefono2)
            telefono = telefono2
            print('Matricula con error: {}, Alumno: {}'.format(matricula, alumno))

    if matricula in matric_yacontactadas:
        print('Matricula antes contactada: {}, alumno: {}'.format(matricula, alumno))
    else:
        mensaje = 'Se acerca la fecha de Extraordinarios. ¿Cómo te encuentras ' + alumno + ' ' + matricula + '? \nTe habla Román Rodríguez Tutor Presencial del CNCI, me comunico contigo debido a que veo que dejaste en ' + tipoExamen + ' la(s) siguiente(s) materia(s): ' + materias +'. \n¿Me gustaría saber si puedo apoyarte con el proceso de regularización? \nTe dejo las fechas siguientes:'
        # mensaje2 = 'Hola, que tal ' + alumno + ' ' + matricula_reprobados + ', soy Román Rodríguez tu Tutor Presencial del CNCI, me comunico contigo debido a que veo que dejaste en ' + tipoExamen + 'la(s) siguiente(s) materia(s): ' + materias +'. ¿Me gustaría saber si puedo apoyarte con el proceso de tus materias en extraordinario?.'
        print('Enviado a: {}, {}'.format(matricula, alumno))
        # wpp.sendwhat_image(telefono, imagen, mensaje, 25, True)

# Consultas

In [ ]:
extras[extras['Matricula']== 'BNL088083']

In [ ]:
database[database['Matricula']== 'BNL088083']

Avances nuevos

In [ ]:
def extras_AF(destinatario):
    from email.message import EmailMessage
    import smtplib

    remitente = str(input("Correo?: ")) 
    password = str(input("Contraseña?: "))

    mensaje = "¡Hola, mundo!"
    email = EmailMessage()
    email["From"] = remitente
    email["To"] = destinatario
    email["Subject"] = "Correo de prueba"
    email.set_content(mensaje)
    # smtp = smtplib.SMTP("smtp-mail.outlook.com", port=587)
    smtp = smtplib.SMTP("smtp.office365.com", port=587)
    smtp.starttls()
    smtp.login(remitente, password)
    smtp.sendmail(remitente, destinatario, email.as_string())